# Core

> The Plash CLI tool

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser, json, httpx, io, tarfile
from pathlib import Path
from uuid import uuid4
from time import time, sleep

import io, re, sys, tarfile, tomllib

## Helpers

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash_config.json'

In [ ]:
#| export
def get_client(cookie_file):
    client = httpx.Client()
    if not cookie_file.exists():
        raise FileNotFoundError("Plash config not found. Please run plash_login and try again.")
    cookies = Path(cookie_file).read_json()
    client.cookies.update(cookies)
    return client

In [ ]:
#| export
def mk_auth_req(url:str, method:str='get', **kwargs): return getattr(get_client(PLASH_CONFIG_HOME), method)(url, **kwargs)

In [ ]:
#| export
def get_app_id(path:Path):
    plash_app = Path(path) / '.plash'
    if not plash_app.exists(): raise FileNotFoundError(f"File not found: {plash_app=}")
    return parse_env(fn=plash_app)['PLASH_APP_ID']

In [ ]:
#| export
def endpoint(path, local, port=None):
    p = "http" if local else "https"
    d = f"localhost:{port}" if local else "pla.sh"
    return f"{p}://{d}{path}"

In [ ]:
#| export
def is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

In [ ]:
#| export
def _tarz(path # Path to directory containing FastHTML app
                      )->io.BytesIO: # Buffer of tar directory
    "Creates a tar archive of a directory, excluding files based on is_included"
    buf = io.BytesIO()
    files = L(Path(path).iterdir()).filter(is_included)

    with tarfile.open(fileobj=buf, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
    buf.seek(0)
    return buf, len(files)

In [ ]:
#| export
def validate_app(path):
    "Validates that the app in the directory `path` is deployable as a FastHTML app"
    print("Analyzing project structure...")

    main_file = Path(path) / "main.py"
    if not main_file.exists():
        print('[red bold]ERROR: Your FastHTML app must have a main.py[/red bold]')
        print(f'Your path is: [bold]{path}[/bold]')
        sys.exit(1)

## Plash - login

In [ ]:
#| export
def poll_cookies(paircode, local, port=None, interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    client = httpx.Client()
    url = endpoint(f"/cli_token?paircode={paircode}",local,port)
    while time()-start < timeout:
        resp = client.get(url).raise_for_status()
        if resp.text.strip(): return dict(client.cookies)
        sleep(interval)
        
@call_parse
def login(
    local:bool=False,  # Local dev
    port:int=5002      # Port for local dev
):
    "Authenticate CLI with server and save config"
    paircode = secrets.token_urlsafe(16)
    login_url = httpx.get(endpoint(f"/cli_login?paircode={paircode}",local,port)).text
    print(f"Opening browser for authentication:\n{login_url}\n")
    webbrowser.open(login_url)
    
    cookies = poll_cookies(paircode, local, port)
    if cookies:
        Path(PLASH_CONFIG_HOME).write_text(json.dumps(cookies))
        print(f"Authentication successful! Config saved to {PLASH_CONFIG_HOME}")
    else: print("Authentication timed out.")

## App - deploy

In [ ]:
#| export
pat = r'(?m)^# /// (?P<type>[a-zA-Z0-9-]+)$\s(?P<content>(^#(| .*)$\s)+)^# ///$'

def _deps(script: bytes | str) -> dict | None:
    '''Get the dependencies from the script. From: https://peps.python.org/pep-0723/'''
    name = 'script'
    if isinstance(script, bytes): script = script.decode('utf-8')
    matches = L(re.finditer(pat, script)).filter(lambda m: m.group('type') == name)
    if len(matches) > 1: raise ValueError(f'Multiple {name} blocks found')
    elif len(matches) == 1:
        content = ''.join(line[2:] if line.startswith('# ') else line[1:]
                          for line in matches[0].group('content').splitlines(keepends=True))
        return '\n'.join(tomllib.loads(content)['dependencies'])
    else: return None

In [ ]:
#| export
def _tarz(path)->io.BytesIO: # Buffer of tar directory
    "Creates a tar archive of a directory, excluding files based on is_included"
    tarz = io.BytesIO()
    files = L(path if path.is_file() else Path(path).iterdir()).filter(is_included)

    with tarfile.open(fileobj=tarz, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
        if path.is_file():
            if deps := _deps(path.read_bytes()):
                info = tarfile.TarInfo('requirements.txt')
                info.size = len(deps)
                tar.addfile(info, io.BytesIO(deps.encode('utf-8')))
    tarz.seek(0)
    return tarz, len(files)

In [ ]:
#| export
@call_parse
def deploy(
    path:Path=Path('.'), # Path to project
    app_id:str=None,     # App ID that will be used as the subdomain in plash
    local:bool=False,    # Local dev
    port:int=5002):      # Port for local dev
    '🚀 Ship your app to production'
    print('Initializing deployment...')
    if path.is_file():
        tarz, _ = _tarz(path)
        aid = app_id or f'fasthtml-app-{str(uuid4())[:8]}'
    else:
        validate_app(path)
        tarz, _ = _tarz(path)
        plash_app = Path(path) / '.plash'
        if not app_id and not plash_app.exists():
            # Create the .plash file and write the app name
            plash_app.write_text(f'export PLASH_APP_ID=fasthtml-app-{str(uuid4())[:8]}')
        aid = app_id or parse_env(fn=plash_app)['PLASH_APP_ID']
    
    resp = mk_auth_req(endpoint("/upload",local,port), "post", files={'file': tarz}, timeout=300.0, data={'aid': aid})
    if resp.status_code == 200: 
        print('✅ Upload complete! Your app is currently being built.')
        if local: print(f'It will be live at http://{aid}.localhost')
        else: print(f'It will be live at https://{aid}.pla.sh')
    else:
        print(f'Failure {resp.status_code}')
        print(f'Failure {resp.text}')

## App - view

In [ ]:
#| export
@call_parse
def view(
    path:Path=Path('.'), # Path to project
    local:bool=False,    # Local dev
):
    aid = get_app_id(path)
    if local: url = f"http://{aid}.localhost"
    else: url = f"https://{aid}.pla.sh"
    print(f"Opening browser to view app :\n{url}\n")
    webbrowser.open(url)

## App - delete

In [ ]:
#| export
@call_parse
def delete(
    path:Path=Path('.'), # Path to project
    force:bool=False,    # Skip confirmation prompt
    local:bool=False,    # Local dev
    port:int=5002):      # Port for local dev
    'Delete your deployed app'
    aid = get_app_id(path)
    if not force:
        confirm = input(f"Are you sure you want to delete app '{aid}'? This action cannot be undone. [y/N]: ")
        if confirm.lower() not in ['y', 'yes']:
            print("Deletion cancelled.")
            return
    
    print(f"Deleting app '{aid}'...")
    r = mk_auth_req(endpoint(f"/delete?aid={aid}",local,port), "delete")
    return r.text

## App - start stop

In [ ]:
#| export
def endpoint_func(endpoint_name):
    'Creates a function for a specific API endpoint'
    @call_parse
    def func(
        path:Path=Path('.'), # Path to project
        local:bool=False,    # Local dev
        port:int=5002):      # Port for local dev
        aid = get_app_id(path)
        r = mk_auth_req(endpoint(f"{endpoint_name}?aid={aid}", local, port))
        return r.text
    
    # Set the function name and docstring
    func.__name__ = endpoint_name
    func.__doc__ = f"Access the '{endpoint_name}' endpoint for your app"
    
    return func

# Create endpoint-specific functions
stop = endpoint_func('/stop')
start = endpoint_func('/start')

## App - logs

In [ ]:
#| export
log_modes = str_enum('log_modes', 'build', 'app')

In [ ]:
#| export
@call_parse
def logs(
    path:Path=Path('.'),    # Path to project
    mode:log_modes='build', # Choose between build or app logs
    tail:bool=False,        # Tail the logs
    local:bool=False,       # Local dev
    port:int=5002):         # Port for local dev
    'Prints the logs for your deployed app'
    aid = get_app_id(path)
    if tail:
        text = ''
        while True:
            try:
                r = mk_auth_req(endpoint(f"/logs?aid={aid}&mode={mode}",local,port))
                if r.status_code == 200:
                    print(r.text[len(text):], end='') # Only print updates
                    text = r.text
                    sleep(1)
                else:
                    print(f"Error: {r.status_code}")
            except KeyboardInterrupt:
                return "\nExiting"
    r = mk_auth_req(endpoint(f"/logs?aid={aid}&mode={mode}",local,port))
    return r.text

## App - download

In [ ]:
#| export
@call_parse
def download(
    path:Path=Path('.'),                # Path to project
    save_path:Path=Path("./download/"), # Save path (optional) 
    local:bool=False,                   # Local dev
    port:int=5002,                      # Port for local dev
    ):
    'Download your deployed app.'
    aid = get_app_id(path)
    try: save_path.mkdir(exist_ok=False)
    except: print(f"ERROR: Save path ({save_path}) already exists. Please rename or delete this folder to avoid accidental overwrites.")
    else:
        response = mk_auth_req(endpoint(f'/download?aid={aid}', local, port)).raise_for_status()
        file_bytes = io.BytesIO(response.content)
        with tarfile.open(fileobj=file_bytes, mode="r:gz") as tar: tar.extractall(path=save_path)
        print(f"Downloaded your app to: {save_path}")

## Export

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()